In [80]:
import pandas as pd

In [81]:
df = pd.read_csv('data/apartment_data_raw.csv') # Main raw dataset
test_df = pd.read_csv('data/apartment_data_raw.csv') # For showcasing changes done when using inplace=True

# Cleaning The Raw Data

Below, the data that has been fetched from _hemnet.se_ will be cleaned up, this will include (no specific meaning of the order):

__1. Removing unnecessary columns and Updating column names__

__2. Rearranging column order__

__3. Converting datatypes__

__4. Removing redundant text__

__5. Editing damaged rows, eg: NaN's, misspelled strings, bad numbers__

It will mainly be done using pandas, each step will be documented and also kept in functions as much as possible for future usage.

All the steps in this cleaning process are crucial for further analyzing the data, storing for example the Fee column as a string won't do us much good.

In [82]:
test_df.head(1) # Using head(1) here to get an overview what columns we want to drop, we use test_df since it will also be used as a comparison in the end of the cleaning process.

,Adress,Area,Date,Fee,Price,Price Per SQM,Size,index,timestamp,timestampString,uid,url,url_uid
0,"Götgatan 128, 3tr","LägenhetLägenhet Södermalm,...",Såld 3 februari 2023,2 798 kr/mån,Slutpris 4 850 000 kr,74 615 kr/m²,65 m² 2 rum,1,1675427967343,"Fri, 03 Feb 2023 12:39:27 GMT",bbcR8OjQt5f0EriklLFM,https://www.hemnet.se/salda/bostader?location_...,1


## 1. Removing unnecessary columns and Updating column names

This dataset has been fetched to analyze two room apartments in popular areas around and in Stockholm City.

Looking at the columns, we see that:
* index
* timestamp
* timestampString
* uid
* url
* url_uid

are all redundant columns for this project and is data that followed during the scraping process.

In [83]:
df.drop(
    ['index', 'timestamp', 'timestampString', 'uid', 'url', 'url_uid'],
    axis=1,
    inplace=True
)
# Dropping columns by passing in a list of the column names
# axis=1 stands for columns
# inplace=True to update the dataframe in place instead of returning a copy of the object.

Now, let's change the name of the column to more 'code friendly' names

In [84]:
df.columns = ['adress', 'area', 'date_sold', 'monthly_fee', 'price', 'price_per_sqm', 'size_in_sqm']

In [85]:
df.head(1)

,adress,area,date_sold,monthly_fee,price,price_per_sqm,size_in_sqm
0,"Götgatan 128, 3tr","LägenhetLägenhet Södermalm,...",Såld 3 februari 2023,2 798 kr/mån,Slutpris 4 850 000 kr,74 615 kr/m²,65 m² 2 rum


This looks better, the columns needed for the analysis is much clearer now. 

We still have to clean the row data, but now we also have the knowledge as to how we want to lay that out.

 
 
______________________________________________________________________________________________________________________________
 
 

## 2. Rearranging column order

Not the most important step, but it's nice to have the columns structured in a way that you want it to be.

In [86]:
df = df.loc[:, ['area', 'adress', 'price', 'price_per_sqm', 'monthly_fee', 'size_in_sqm', 'date_sold']]

In [87]:
df.head(1)

,area,adress,price,price_per_sqm,monthly_fee,size_in_sqm,date_sold
0,"LägenhetLägenhet Södermalm,...","Götgatan 128, 3tr",Slutpris 4 850 000 kr,74 615 kr/m²,2 798 kr/mån,65 m² 2 rum,Såld 3 februari 2023


Sweet! We have created a new layout for the dataset, lets starting looking at cleaning the actual data.

## 3. Converting datatypes

Let's have a look at what datatypes we are working with in each column:

In [88]:
df.dtypes

area             object
adress           object
price            object
price_per_sqm    object
monthly_fee      object
size_in_sqm      object
date_sold        object
dtype: object

Object in this case stands for string, which is something we don't want for some of the columns.

More specifically, we want to convert the necessary columns like this:
* __price__: String -> Integer
* __price_per_sqm__: String -> Integer
* __monthly_fee__: String -> Float
* __size_in_sqm__: String -> Integer
* __date_sold__: String -> datetime64 (using datetime from NumPy)

Doing this means we also have to change the strings to only contain numbers.

In [89]:
import re

Trying to convert these columns with the str.replace() method did not work correctly so instead we have to use the re module.

In [90]:
def string_to_integer(df_column):
    # Use regular expression to find all numeric values in the string and convert them to a list of strings
    df_column = df_column.apply(lambda x: [str(i) for i in re.findall(r'\d+', x)]) 
    
    # Concatenate the list of strings into one string and then convert it to an integer
    df_column = df_column.apply(lambda x: int(''.join(map(str, x))))
    
    # Return the resulting column
    return df_column

In [91]:
def float_to_integer(df_column):
    # Convert floats to strings
    df_column = df_column.apply(lambda x: str(x)) 
    
    # Keep only the numeric values as a string
    df_column = df_column.apply(lambda x: ''.join(re.findall(r'\d+', x))) 
    
    # Convert the string to integer or return None if not a digit
    df_column = df_column.apply(lambda x: int(x) if x.isdigit() else None) 
    
    return df_column

_string_to_integer_ takes in a pandas dataframe column as input and returns a new pandas series where each value in the column has been transformed to an integer. The function first converts the input to a list of strings where each string only contains numeric characters using the re.findall method. Then, it converts the list to a single integer value using the int function.

_float_to_integer_ also takes in a pandas dataframe column as input and returns a new pandas series where each value has been transformed to an integer. The function first converts the input values to strings, then it uses the re.findall method to extract all numeric characters and concatenate them into a single string. Finally, it uses the int function to convert the string to an integer. If the string does not represent a digit, the function returns None instead.

Now we can simply call the functions and update the columns

In [92]:
df['price'] = string_to_integer(df['price'])
df['price_per_sqm'] = string_to_integer(df['price_per_sqm'])
df['monthly_fee'] = float_to_integer(df['monthly_fee'])
df['size_in_sqm'] = string_to_integer(df['size_in_sqm'])

In [93]:
df.head(5)

,area,adress,price,price_per_sqm,monthly_fee,size_in_sqm,date_sold
0,"LägenhetLägenhet Södermalm,...","Götgatan 128, 3tr",4850000,74615,2798.0,652,Såld 3 februari 2023
1,"LägenhetLägenhet Årsta, ...",Möckelvägen 32,2650000,63095,3281.0,422,Såld 3 februari 2023
2,LägenhetLägenhet Södermalm ...,Heleneborgsgatan 30,4500000,97826,2451.0,462,Såld 2 februari 2023
3,"LägenhetLägenhet Östermalm,...","Körsbärsvägen 10, 5tr",5025000,94811,2664.0,532,Såld 2 februari 2023
4,LägenhetLägenhet Vasastan H...,"Anna Steckséns Gata 15, 10 tr",4900000,128947,2387.0,382,Såld 2 februari 2023


In [94]:
df['date_sold'] = df['date_sold'].str.replace('Såld', '').str.lstrip()

In [95]:
df['date_sold'] = df['date_sold'].str.replace('januari','1')
df['date_sold'] = df['date_sold'].str.replace('februari', '2')
df['date_sold'] = df['date_sold'].str.replace('mars', '3')
df['date_sold'] = df['date_sold'].str.replace('april', '4')
df['date_sold'] = df['date_sold'].str.replace('maj', '5')
df['date_sold'] = df['date_sold'].str.replace('juni', '6')
df['date_sold'] = df['date_sold'].str.replace('juli','7')
df['date_sold'] = df['date_sold'].str.replace('augusti', '8')
df['date_sold'] = df['date_sold'].str.replace('september', '9')
df['date_sold'] = df['date_sold'].str.replace('oktober', '10')
df['date_sold'] = df['date_sold'].str.replace('november', '11')
df['date_sold'] = df['date_sold'].str.replace('december', '12')

In [96]:
import datetime

In [97]:
df['date_sold'] = df['date_sold'].str.replace(' ', '/')

In [98]:
df['date_sold']

0        3/2/2023
1        3/2/2023
2        2/2/2023
3        2/2/2023
4        2/2/2023
          ...    
2452    12/2/2022
2453    12/2/2022
2454    11/2/2022
2455    11/2/2022
2456    11/2/2022
Name: date_sold, Length: 2457, dtype: object

In [99]:
df['date_sold'] = pd.to_datetime(df['date_sold'], format='%d/%m/%Y')

In [101]:
df.head(1)

,area,adress,price,price_per_sqm,monthly_fee,size_in_sqm,date_sold
0,"LägenhetLägenhet Södermalm,...","Götgatan 128, 3tr",4850000,74615,2798.0,652,2023-02-03


In [100]:
df.dtypes

area                     object
adress                   object
price                     int64
price_per_sqm             int64
monthly_fee             float64
size_in_sqm               int64
date_sold        datetime64[ns]
dtype: object

Great! We have the correct datatypes for each column and are ready to go to the next part of the cleaning process.

## 4. Removing redundant text

In [116]:
df['area'] = df['area'].str.replace('LägenhetLägenhet ', '') # Replacing weird words from scraping process with blank

In [117]:
df.head()

,area,adress,price,price_per_sqm,monthly_fee,size_in_sqm,date_sold
0,Södermalm,"Götgatan 128, 3tr",4850000,74615,2798.0,652,2023-02-03
1,Årsta,Möckelvägen 32,2650000,63095,3281.0,422,2023-02-03
2,Södermalm,Heleneborgsgatan 30,4500000,97826,2451.0,462,2023-02-02
3,Östermalm,"Körsbärsvägen 10, 5tr",5025000,94811,2664.0,532,2023-02-02
4,Vasastan,"Anna Steckséns Gata 15, 10 tr",4900000,128947,2387.0,382,2023-02-02


In [118]:
df['area'] = df['area'].str.extract(r'(Södermalm|Årsta|Östermalm|Vasastan|Fredhäll)') # Keeps only the areas we are interested in and removes the more specific description

Let's compare the raw dataset with our cleaned to see how far we have come.

In [122]:
df.head()

,area,adress,price,price_per_sqm,monthly_fee,size_in_sqm,date_sold
0,Södermalm,"Götgatan 128, 3tr",4850000,74615,2798.0,652,2023-02-03
1,Årsta,Möckelvägen 32,2650000,63095,3281.0,422,2023-02-03
2,Södermalm,Heleneborgsgatan 30,4500000,97826,2451.0,462,2023-02-02
3,Östermalm,"Körsbärsvägen 10, 5tr",5025000,94811,2664.0,532,2023-02-02
4,Vasastan,"Anna Steckséns Gata 15, 10 tr",4900000,128947,2387.0,382,2023-02-02


In [123]:
test_df.head()

,Adress,Area,Date,Fee,Price,Price Per SQM,Size,index,timestamp,timestampString,uid,url,url_uid
0,"Götgatan 128, 3tr","LägenhetLägenhet Södermalm,...",Såld 3 februari 2023,2 798 kr/mån,Slutpris 4 850 000 kr,74 615 kr/m²,65 m² 2 rum,1,1675427967343,"Fri, 03 Feb 2023 12:39:27 GMT",bbcR8OjQt5f0EriklLFM,https://www.hemnet.se/salda/bostader?location_...,1
1,Möckelvägen 32,"LägenhetLägenhet Årsta, ...",Såld 3 februari 2023,3 281 kr/mån,Slutpris 2 650 000 kr,63 095 kr/m²,42 m² 2 rum,2,1675427967343,"Fri, 03 Feb 2023 12:39:27 GMT",uk6ZHHepLVJmFTqQHy0Z,https://www.hemnet.se/salda/bostader?location_...,1
2,Heleneborgsgatan 30,LägenhetLägenhet Södermalm ...,Såld 2 februari 2023,2 451 kr/mån,Slutpris 4 500 000 kr,97 826 kr/m²,46 m² 2 rum,3,1675427967343,"Fri, 03 Feb 2023 12:39:27 GMT",cjyLRjgOnOMK82EonWDr,https://www.hemnet.se/salda/bostader?location_...,1
3,"Körsbärsvägen 10, 5tr","LägenhetLägenhet Östermalm,...",Såld 2 februari 2023,2 664 kr/mån,Slutpris 5 025 000 kr,94 811 kr/m²,53 m² 2 rum,4,1675427967343,"Fri, 03 Feb 2023 12:39:27 GMT",Q6siuV1K48MIikhcKHvj,https://www.hemnet.se/salda/bostader?location_...,1
4,"Anna Steckséns Gata 15, 10 tr",LägenhetLägenhet Vasastan H...,Såld 2 februari 2023,2 387 kr/mån,Slutpris 4 900 000 kr,128 947 kr/m²,38 m² 2 rum,5,1675427967343,"Fri, 03 Feb 2023 12:39:27 GMT",0FTq9jrx3XABUYK6ZuuZ,https://www.hemnet.se/salda/bostader?location_...,1
